In [1]:
from dino_utils import calculate_embeddings

Using cuda as device
Using cuda as device
Using cuda as device


/pfs/work7/workspace/scratch/tu_zxmav84-thesis/Thesis/dinov2/dinov2/layers/swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
/pfs/work7/workspace/scratch/tu_zxmav84-thesis/Thesis/dinov2/dinov2/layers/attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
/pfs/work7/workspace/scratch/tu_zxmav84-thesis/Thesis/dinov2/dinov2/layers/block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")


In [2]:
import pickle
import platform
import os
if platform.system() == 'Darwin':
    DATA_PATH = "/Users/maltegenschow/Documents/Uni/Thesis/Data.nosync"
    ROOT_PATH = "/Users/maltegenschow/Documents/Uni/Thesis/Thesis"
elif platform.system() == 'Linux':
    DATA_PATH = "/pfs/work7/workspace/scratch/tu_zxmav84-thesis/Data.nosync"
    ROOT_PATH = "/pfs/work7/workspace/scratch/tu_zxmav84-thesis/Thesis"

current_wd = os.getcwd()

In [3]:
import torch
import os
from glob import glob
from tqdm import tqdm
import numpy as np

### Compute Embeddings for Real Images

In [4]:
# Real Images
input_images_path = f"{DATA_PATH}/Zalando_Germany_Dataset/dresses/images/e4e_images/all/"
save_path = f"{DATA_PATH}/Models/Assessor/DinoV2/Embeddings/real_images_embeddings.pt"
calculate_embeddings(input_images_path, save_path)

Embeddings for /pfs/work7/workspace/scratch/tu_zxmav84-thesis/Data.nosync/Zalando_Germany_Dataset/dresses/images/e4e_images/all/ already calculated


### Compute Embeddings for e4e Inversions from saved images

In [5]:
# e4e Reconstructions
input_images_path = f"{DATA_PATH}/Generated_Images/e4e/00005_snapshot_1200/"
save_path = f"{DATA_PATH}/Models/Assessor/DinoV2/Embeddings/e4e_embeddings_from_saved.pt"
calculate_embeddings(input_images_path, save_path)

Embeddings for /pfs/work7/workspace/scratch/tu_zxmav84-thesis/Data.nosync/Generated_Images/e4e/00005_snapshot_1200/ already calculated


### Calculate Embeddings for PTI Images

In [6]:
skus = glob(f'{DATA_PATH}/Generated_Images/PTI/*.jpg')
skus = [elem.split('/')[-1].split('.')[0] for elem in skus]

In [7]:
save_path = f"{DATA_PATH}/Models/Assessor/DinoV2/Embeddings/pti_embeddings.pt"
calculate_embeddings(skus, save_path, generator_type='PTI', generator=None)

Embeddings for ['TRU21C01C-C11', 'UP121C1FA-K11', 'VM321C0NX-K11', 'AW721C073-M11', 'H0721C02D-I11', 'DI121C0EB-Q11', 'KA321C141-Q12', 'WG021C11C-K11', 'LGN21C010-J12', '02R21C014-Q11', 'LE221C042-K11', 'KA321C0NU-Q11', 'NII21C04V-Q11', 'ADK21C03N-N11', 'W2K21C00S-Q11', 'SO221C1BS-H11', 'VE121C4AG-A11', 'G8721C009-Q11', 'VE121C45I-Q11', 'VK121C03Z-A11', 'BEJ21C1CS-B11', 'AN621C20L-Q11', 'YA021C0B9-Q11', 'UP121C1D6-P11', 'A0Q21C0V9-T11', 'V1021C2H4-Q11', 'L2521C0H4-H11', 'ON321C3MP-M11', 'HU721C0QX-J11', 'LUQ21C06Y-Q11', 'SA321C0K3-Q11', 'CUI21C02Y-Q11', 'AN621C21Q-Q11', 'STH21C15X-K11', 'SO221C1DT-K11', 'V1021C43W-J11', 'NX321C3Y2-K12', 'M4R21C0FQ-K11', '4BE21C0BL-K12', 'GP021C0R7-B11', 'C2P21C09E-Q11', 'WG021C03X-M12', 'WAH21C043-Q11', 'M5921C194-B11', 'GE221C0GO-Q11', 'M5O21C093-K12', 'M6429F16F-I11', 'RI521C085-T11', 'L0J21C0L0-Q11', 'E3221C008-H11', 'T6P21C000-B11', 'MUO21C02J-A11', 'M1X21C03Y-O11', 'TA021C0DV-Q11', 'KRG21C03D-K12', 'SG721C0YN-K11', '2CR21C019-C11', 'VM321C119-I11'

### Compute Embeddings for Hyperstyle Images

In [8]:
os.chdir(f'{ROOT_PATH}/2_Inversion/hyperstyle/')
from hyperstyle_utils import load_hyperstyle, load_generator_inputs, generate_hyperstyle
os.chdir(f"{current_wd}")

In [9]:
skus = glob(f"{DATA_PATH}/Zalando_Germany_Dataset/dresses/images/e4e_images/all/*.jpg")
skus = [elem.split('/')[-1].split('.')[0] for elem in skus]

In [10]:
G = load_hyperstyle()
save_path = f"{DATA_PATH}/Models/Assessor/DinoV2/Embeddings/hyperstyle_embeddings.pt"
calculate_embeddings(skus, save_path, generator_type='hyperstyle', generator=G)

Loading HyperStyle from checkpoint: /pfs/work7/workspace/scratch/tu_zxmav84-thesis/Data.nosync/Models/hyperstyle/00005_snapshot_1200_restyle_77000/resume/checkpoints/best_model.pt
Loading pretrained W encoder...
Loading ReStyle e4e from checkpoint: /pfs/work7/workspace/scratch/tu_zxmav84-thesis/Data.nosync/Models/restyle/00005_snapshot_1200/checkpoints/best_model.pt
Model successfully loaded!
Embeddings for ['CE321C0DU-K11', 'FAB21C095-K11', 'IV321C0E5-Q11', 'NN121C0H6-A11', 'CO121C1C1-Q12', 'ONR21C00B-N11', 'V1021C48D-H11', 'GAG21C039-Q11', 'SHI21C015-G11', 'VE121C472-Q11', 'ONR21C005-J12', 'TRU21C01C-C11', 'NX321C45N-Q11', 'BY221C0I0-B11', 'KAN21C043-K12', 'GS121C0GU-K11', 'ARC21C07B-K11', 'DD121C011-T11', 'ON321C2SI-K12', 'UP121C1FV-Q11', 'BY221C0EA-T11', 'F2121C07A-M11', 'V1021C2W7-M11', 'VM321C0T0-A11', 'R5921C0I1-C11', 'EA721C016-Q11', 'MAL21C0JX-M11', 'BQ021C050-T11', 'V1021C0VX-O17', 'KRG21C04Q-M11', 'IZ021C04J-J11', 'TWG21C07Q-Q11', 'CU221C0C6-M12', 'C2P21C021-A11', 'ONR29F02O